# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [23]:
# initial imports
import os
import pandas as pd

In [24]:
import panel as pn
pn.extension('plotly')
import hvplot.pandas
# Initialize the Panel Extensions (for Plotly)
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv



In [25]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [26]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [27]:
# Define Panel Visualization Functions
def plot_housing_units_year():
    """Housing Units Per Year."""
housing_units = sfo_data.groupby('year').mean()
housing_units_year = housing_units['housing_units']
housing_units_year_grp = housing_units_year.sort_values()
plot_housing_units_year = housing_units_year_grp.hvplot.bar(
    label = 'Housing Units in San Francisco from 2010 to 2016', 
    ylim = (370000, 385000), 
    width = 400, 
    rot = 90)

In [28]:
def plot__gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
gross_rent_grp= sfo_data.groupby('year').mean()
gross_rent = gross_rent_grp[["gross_rent", "sale_price_sqr_foot"]]
plot_gross_rent = gross_rent_grp.hvplot(
    label = 'Average Gross Rent in San Francisco', 
    y = 'gross_rent', 
    x = 'year',
    width = 400)

In [29]:
def plot__sales_price():
    """Average Sales Price Per Year."""
    
plot_sales_price = gross_rent_grp.hvplot(
    label = 'Average Sales Price per Square Foot in San Francisco', 
    y = 'sale_price_sqr_foot', 
    x = 'year',
    width = 400)

In [30]:
def neighborhood_grp_plot2():
    """Average Prices by Neighborhood."""
    
neighborhood_grp= sfo_data.groupby(['year','neighborhood']).mean()

average_price_by_neighborhood = neighborhood_grp.reset_index()
neighborhood_grp_plot = average_price_by_neighborhood.hvplot(
    groupby = 'neighborhood', 
    y = 'sale_price_sqr_foot', 
    x = 'year')
neighborhood_grp_plot2 = neighborhood_grp.hvplot(
    groupby = 'neighborhood', 
    y = 'sale_price_sqr_foot', 
    x = 'year')

In [51]:
def neighborhood_ten_plot():
    """Top 10 Most Expensive Neighborhoods."""
    
neighborhood_sales = sfo_data.groupby(['neighborhood']).mean()
neighborhood_ten = neighborhood_sales.nlargest(10, 'sale_price_sqr_foot')
neighborhood_tenR = neighborhood_ten.reset_index()
neighborhood_ten_plot = neighborhood_ten.hvplot.bar(
    y = 'sale_price_sqr_foot', 
    x = 'neighborhood', 
    rot = 90, 
    title = "Top 10 Expensive Neighborhoods in SFO")


In [61]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
plot_parallel_coordinates = px.parallel_coordinates(neighborhood_tenR, color="sale_price_sqr_foot", labels={"sale_price_sqr_foot": "Sale Price per Square Foot",
                "housing_units": "Housing Units", "gross_rent": "Gross Rent" })

In [60]:
def parallel_categories():
    """Parallel Categories Plot."""
    
plot_parallel_categories = px.parallel_categories(
    neighborhood_tenR,
    
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
       "sale_price_sqr_foot": "Sale Price per Square Foot",
                "housing_units": "Housing Units", "gross_rent": "Gross Rent" 
    },
)

In [54]:
def neighborhoodsalesmap():
    """Neighborhood Map"""
    
neighborhood_sales2 = neighborhood_sales.reset_index()
mapped_neighborhood_sales = pd.concat([neighborhood_sales2,df_neighborhood_locations], axis="columns", join="inner")
cleanmapped_neighborhood_sales = mapped_neighborhood_sales.drop(columns = "Neighborhood")
neighborhoodsalesmap = px.scatter_mapbox(
    cleanmapped_neighborhood_sales,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color = "neighborhood"
)



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [57]:
plot_Yearly_Market = pn.Column(
    "Neighborhood Analysis", plot_housing_units_year, plot_gross_rent, plot_sales_price
)
plot_Yearly_Market

Column
    [0] Markdown(str)
    [1] HoloViews(Bars)
    [2] HoloViews(Curve)
    [3] HoloViews(Curve)

In [63]:
plot_neighborhood_analysis = pn.Column(
    "Neighborhood Analysis", neighborhood_grp_plot2, neighborhood_ten_plot
)
plot_neighborhood_analysis

Column
    [0] Markdown(str)
    [1] Row
        [0] HoloViews(DynamicMap)
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
            [1] VSpacer()
    [2] HoloViews(Bars)

In [64]:
parallel_analysis = pn.Column(
    "Parallel Plot Analysis",plot_parallel_coordinates, plot_parallel_categories,
)
parallel_analysis

Column
    [0] Markdown(str)
    [1] Plotly(Figure)
    [2] Plotly(Figure)

In [67]:
panel = pn.Tabs(
    ("Welcome", neighborhoodsalesmap ),
    ("Yearly Market Analysis", plot_Yearly_Market),
    ("Neighborhood Analysis",plot_neighborhood_analysis),
    ("Parallel Plots Analysis", parallel_analysis)
)
panel

Tabs
    [0] Plotly(Figure, name='Welcome')
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)

In [ ]:
## Serve the Panel Dashboard

In [68]:
panel.servable()

Tabs
    [0] Plotly(Figure, name='Welcome')
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)